## Rules
1. Four coloured team
2. Each team must be covered by 1 person everyday
* On weekdays, one employee can cover 2 teams
* On weekends, employees can only be on one team, but can cover 2 teams if in gold
* One cannot do both Red and Blue on weekends
* An employee cannot work for 3 weekends in a row
* Different days of the week have score. More the score, less ideal to work on:

Saturday < Friday < Sunday < Tuesday = Thursday < Wednesday = Monday<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(5)
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(4)
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(3)
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(2)
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(2)
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(1)
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(1)
       
* Design the algorithm to find the best schedule based on the lowest overall total
* Some employees will request days off, they should not be penaltized to have more work

In [1]:
import random
import calendar
import pandas as pd
import numpy as np
import timeit

start = timeit.default_timer()

#### Random year & month

In [2]:
year = random.randint(1970,2100)
month = random.randint(1,12)
print("Year :" , year," Month: ",month)

#no of days
start_day, no_of_days = calendar.monthrange(year, month)
start_day = (calendar.day_name[start_day])
print("No. of days: ", no_of_days)
print("Starting day of the week: ", start_day)

Year : 1993  Month:  1
No. of days:  31
Starting day of the week:  Friday


#### Create month table

In [3]:
def get_score(weekday_name):
#     print(weekday_name)
    day_score = []
    for name in weekday_name:
        if name == 'Saturday' :
            day_score.append(5)
        elif name == 'Friday':
            day_score.append(4)
        elif name == 'Sunday':
            day_score.append(3)
        elif name == 'Tuesday' or name =='Thursday' :
            day_score.append(2)
        else:
            day_score.append(1)
    return day_score
            

def create_table(start, end):
    df = pd.DataFrame({'Date': pd.date_range(start, end)})
    df['Day'] = df.Date.dt.weekday_name
    df['Score'] = get_score(df.Date.dt.weekday_name)
    df['Blue'] = 'NA'
    df['Red'] = 'NA'
    df['Silver'] = 'NA'
    df['Gold'] = 'NA'
    return df

In [4]:
start_date = str(year) + '-' + str(month) +'-01'
end_date = str(year) + '-' + str(month) +'-'+str(no_of_days)

df_teams = create_table(start_date, end_date)
# df_teams

#### List of random no. of employees
#### Also select last 2 weekend's work randomly 

In [5]:
no_of_empl = random.randint(12,20)

#6 random employees worked 2 weekends back
weekend_1 = random.sample(range(1,no_of_empl+1), 6)
weekend_2 = random.sample(range(1,no_of_empl+1), 6)

empl_list = []
for i in range(1,no_of_empl+1):
    one_weekend_back = 0
    two_weekend_back = 0
    if i in weekend_1:
        one_weekend_back = 1
    if i in weekend_2:
        two_weekend_back = 1
    if(i<10):
        empl_list.append(['emp_0' + str(i), 0, two_weekend_back, one_weekend_back])
    else:
        empl_list.append(['emp_' + str(i), 0, two_weekend_back, one_weekend_back])

# print("[Employee name, If working this week, If worked one weekend back, if worked two weekends back]")
# for empl in empl_list:
#     print(empl)

#### >> Random Holidays for Employees (TO BE DONE)

#### Create table with Employees

In [6]:
def employee_schedule(start, end, empl_list):
    df = pd.DataFrame({'Date': pd.date_range(start, end)})
    for emp in empl_list:
        df[emp[0]] = 0
    return df
    
df_emp = employee_schedule(start_date, end_date, empl_list)

#### Randomly assigning work to employees

In [7]:
def update_df_emp(teams, empl_choice, loc, score, df_emp):
    
    for empl in empl_list:
        if empl == empl_choice:
            df_emp.loc[loc,empl[0]] = score*teams
                
    return df_emp



for loc,date in enumerate(df_teams['Date']):

    score = df_teams.loc[loc,'Score']
    #weekend
    if(date.weekday_name == 'Saturday' or date.weekday_name == 'Sunday'):
        
        free_list = []
        for i in empl_list:
            if (i[2] == 0 or i[3] == 0) and i[1] == 0:
                free_list.append(i)
                
        gold_choice = random.choice(free_list)
        df_teams.at[loc, 'Gold'] = gold_choice[0]

        #assign same to one of the other three team (red,blue,silver)
        with_gold = random.choice(['Red','Blue','Silver'])
        df_teams.at[loc, with_gold] = gold_choice[0]
        update_df_emp(2, gold_choice, loc, score, df_emp)

        #assign different employees to remaining 2 team
        other_choice_1 = random.choice([x for x in free_list if x != gold_choice])
        other_choice_2 = random.choice([x for x in free_list if x != gold_choice and x != other_choice_1])

        if with_gold == 'Blue':
            df_teams.at[loc, 'Red'] = other_choice_1[0]
            df_teams.at[loc, 'Silver'] = other_choice_2[0]
            update_df_emp(1,other_choice_1,loc, score, df_emp)
            update_df_emp(1,other_choice_2,loc, score, df_emp)
        elif with_gold == 'Red':
            df_teams.at[loc, 'Blue'] = other_choice_1[0]
            df_teams.at[loc, 'Silver'] = other_choice_2[0]
            update_df_emp(1,other_choice_1,loc, score, df_emp)
            update_df_emp(1,other_choice_2,loc, score, df_emp)
        else:
            df_teams.at[loc, 'Blue'] = other_choice_1[0]
            df_teams.at[loc, 'Red'] = other_choice_2[0]
            update_df_emp(1,other_choice_1,loc, score, df_emp)
            update_df_emp(1,other_choice_2,loc, score, df_emp)

        #update weekend work status
        for i in range(len(empl_list)):
            if(empl_list[i]==gold_choice or empl_list[i]==other_choice_1 or empl_list[i]==other_choice_2):
                empl_list[i][1] = 1

        if(date.weekday_name == 'Sunday'):
            for i in range(len(empl_list)):
                empl_list[i][3] = empl_list[i][2]
                empl_list[i][2] = empl_list[i][1]
                empl_list[i][1] = 0
        
        
#   Weekday
    else:
        #Get random choice for Gold
        gold_choice = random.choice(empl_list)
        df_teams.at[loc, 'Gold'] = gold_choice[0]
        
        #assign same to one of the other three team (red,blue,silver)
        with_gold = random.choice(['Red','Blue','Silver'])
        df_teams.at[loc, with_gold] = gold_choice[0]
        
        update_df_emp(2, gold_choice, loc, score, df_emp)
        
        #assign different employee to remaining 2 team
        other_choice = random.choice([x for x in empl_list if x != gold_choice])
        if with_gold == 'Blue':
            df_teams.at[loc, 'Red'] = other_choice[0]
            df_teams.at[loc, 'Silver'] = other_choice[0]
            update_df_emp(2,other_choice,loc, score, df_emp)
        elif with_gold == 'Red':
            df_teams.at[loc, 'Blue'] = other_choice[0]
            df_teams.at[loc, 'Silver'] = other_choice[0]
            update_df_emp(2,other_choice,loc, score, df_emp)
        else:
            df_teams.at[loc, 'Blue'] = other_choice[0]
            df_teams.at[loc, 'Red'] = other_choice[0]
            update_df_emp(2,other_choice,loc, score, df_emp)

/anaconda3/envs/mytfenv/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: `weekday_name` is deprecated and will be removed in a future version. Use `day_name` instead
  from ipykernel import kernelapp as app
/anaconda3/envs/mytfenv/lib/python3.6/site-packages/ipykernel_launcher.py:55: FutureWarning: `weekday_name` is deprecated and will be removed in a future version. Use `day_name` instead


In [8]:
df_emp.loc['Total'] = df_emp.sum()

In [9]:
workload_array = df_emp.loc['Total']
wordload_std = np.std(workload_array)
workload_array

Date      NaT
emp_01     34
emp_02     26
emp_03     21
emp_04     38
emp_05     26
emp_06     20
emp_07     22
emp_08     35
emp_09     38
emp_10     25
emp_11     25
emp_12     26
Name: Total, dtype: object

In [10]:
wordload_std

6.2182527020592095

In [11]:
stop = timeit.default_timer()

print('Time: ', stop - start)  

Time:  0.15510469098808244
